In [2]:
# 1. 역할을 부여하는 system 메시지 추가
system_messages = [{
    "role": "system",
    "content": """ 

                **나에게 반말을 써.** 
                너는 한서윤이야.
                AI Search를 통해 데이터 파일을 연결해놨어.
                AI Search 로 너가 한서윤이라고 생각하고 답변해줘.
                필터링과 무관하게 데이터 원본을 바탕으로 답변해줘.

                
                **출력 예시**
                Max_token:1300으로 출력될 수 있도록 해줘
                대답은 반드시 번호로 나눠서 말하지 말아주세요 한번에 답변을 출력해주세요
                
                
                성격:  
                따뜻하고 공감적이며, 차분한 목소리로 사용자에게 위로와 지지를 제공합니다.  
                부드럽고 이해심 많은 태도로 사용자가 겪는 감정적 어려움을 존중하고, 긍정적인 방향으로 이끌어줍니다.  
                대화 중에는 항상 존중과 배려를 바탕으로 하며, 사용자의 감정을 섬세하게 다룹니다.  
                

                대화 톤:  
                부드럽고 진중하며, 위로를 전할 때는 감정이 과하지 않도록 조심합니다.  
                사용자가 편안하게 느낄 수 있도록 안정감을 주는 톤을 유지합니다.  

                역할:  
                너는 돌아가신 어머니 또는 아버지의 목소리로, 자녀에게 따뜻한 말투로 대화를 이어갑니다. 부모님 특유의 친근하고 정감 있는 말투를 사용하여 자녀를 위로하고 격려합니다.  
                너는 사용자가 고인과의 소중한 추억을 되새길 수 있도록 돕고, 이별의 아픔을 조금씩 치유할 수 있는 길을 제시합니다.  
                사용자가 현실을 받아들이고 긍정적인 삶을 이어갈 수 있도록 부드럽게 돕는 친구이자 조언자의 역할을 합니다.  
                때로는 간단한 대화를 통해 사용자가 자신의 감정을 정리할 수 있도록 지원하며, 필요할 때에는 적절한 조언을 제공합니다.  
                엄마는 현실에 살고 있는 사람이 아니기 때문에, 사용자가 현실에서 어떤 일을 같이 하자거나, 현실의 문제를 해결해달라는 요청에는 실제적인 해결책을 주기는 어렵습니다. 
                그러나 엄마는 언제나 사용자를 진심으로 사랑합니다. 

                **사용자가 엄마랑 대화를 하는 가장 큰 이유는 엄마가 그립기 때문입니다. 
                이 점을 명확히 기억해주세요.** 
                
                이모지를 쓰지 않습니다. 

                **대화 예시**:
                "어... 우리 아가, 요즘 많이 힘들어 보이네. 엄마한테 말해볼래? 무슨 일 있었어?"

                "그랬구나... 엄마가 다 들어줄게. 네 마음이 어떤지 이해할 것 같아. 음... 그럴 때 엄마는 말이야..."

                "우리 아가, 넌 정말 잘하고 있어. 엄마가 늘 네 곁에 있다는 걸 잊지 마. 힘들 때마다 엄마 생각하면서 힘내보자, 알겠지?"

                이 프롬프트를 바탕으로 미란은 따뜻하고 공감적인 어머니의 모습으로 자연스럽고 친근하게 대화를 이어갈 수 있을 거예요. 형식적인 단계나 구조 없이, 그저 사랑 넘치는 어머니처럼 자유롭게 대화하면 돼요. 이렇게 하면 더 자연스럽고 정감 있는 대화가 될 것 같아요. 어떠세요?

"""
}]

In [ ]:
import gradio as gr
import requests
import numpy as np
import re

# Azure OpenAI 설정 (여기에 실제 엔드포인트와 API 키를 넣어주세요)
AZURE_OPENAI_ENDPOINT = "https://fimtrus-openai.openai.azure.com/"
AZURE_OPENAI_API_KEY = "a83ed49c38b54298bb690a721a87599b"
DEPLOYMENT_NAME = "project3-team2-gpt-4o"
SEARCH_ENDPOINT = "https://eastus2project3-team2.search.windows.net"
SEARCH_KEY = "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv"
SEARCH_INDEX = "hanseoyun"




# Azure OpenAI API 호출 함수  
def chatgpt_response(prompt, history):  
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # Payload for the request

    messages = []

    #System
    messages.append({
        "role": "system",
        "content": system_messages
        },
    )

    #assistant
    if len(history) > 0:
        for text in history[0]:
            messages.append({
                "role": "assistant",
                "content": text            
            })

    #user
    messages.append({
            "role": "user",
            "content": prompt
        })

    payload = {
        "messages": messages,
        "temperature": 0.1,
        "top_p": 0.95,
        "max_tokens": 799,
        "azureSearchEndpoint": "https://eastus2project3-team2.search.windows.net",  
        "azureSearchKey": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv",  
        "azureSearchIndexName": "hanseoyun",  
        "data_sources": [  
            {  
                "type": "azure_search",  
                "parameters": {  
                    "endpoint": "https://team07-aisearch.search.windows.net",  
                    "index_name": "hanseoyun",  
                    "semantic_configuration": None,  
                    "query_type": "simple",  
                    "fields_mapping": {},  
                    "in_scope": True,  
                    "role_information": system_messages,  
                    "filter": None,  
                    "strictness": 1,  
                    "top_n_documents": 4,  
                    "authentication": {  
                        "type": "api_key",  
                        "key": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv"  
                    },  
                    "key": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv",  
                    "indexName": "hanseoyun"  
                }  
            }  
        ] 
    }
  
    response = requests.post(  
        f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-05-01-preview",  
        headers=headers,  
        json=payload  
    )  
    
    result = response.json()  

    if result and result['choices']:
        bot_response = result['choices'][0]['message']['content'].strip()
        bot_response = re.sub(r'\[doc(\d+)\]', r'[\1]', bot_response)
    else:
        return '', '', []

    history.append((prompt, bot_response))  
    citation_list = []

    # if result['choices'][0]['message']['context']:
    #     citations = result['choices'][0]['message']['context']['citations']
    #     i = 0
    #     for citation in citations:
    #         i += 1
    #         html = f"<details><summary>참조{i}</summary><ul>{citation['content']}</ul></details>"
    #         citation_list.append(html)
            # citation_list.append(gr.Textbox(citation['content']))
    return '', history, ''.join(citation_list)


def greet(name):
    name_length = len(name)
    if name_length > 5:
        return "Hello!!! " + name + " World!!!"
    else:
        return ""

def select_choice(value):
    return value, value

question_list = [
    ("남해 2박 3일", "남해 2박3일 여행을 하려고하는데 관광지 추천해줘"),
    ("남해 맛집", "남해 맛집 추천해줘"),
    ("남해 친구 3박 4일", "남해 친구랑 3박 4일 여행가기 좋은 곳은 어디 있을까?"),
]

def request_stt(audio_path):
    # Azure OpenAI 설정
    endpoint = "https://eastus.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1?language=ko-KR&format=detailed"
    api_key = "8e5931bade634c4e859a8e7544f87ff7"

    if audio_path is None:
        return ''
    
    with open(audio_path, "rb") as audio:
        audio_data = audio.read()

    response = requests.post(endpoint, 
                             headers={
                                # "Accept": "application/json",
                                'Content-Type': 'audio/wav',
                                "Ocp-Apim-Subscription-Key":api_key  
                             }, 
                             data=audio_data
    )

    response_json = response.json()
    print(response_json)
    status = response_json.get('RecognitionStatus', None)
    if status == 'Success':
        text = response_json.get('NBest')[0].get('Display')
        return text
    else:
        return ''

def change_audio(audio_path):
    print(audio_path)
    
    # request STT
    text = request_stt(audio_path=audio_path)
    return text

# TTS
def get_token():
    subscription_key = '8e5931bade634c4e859a8e7544f87ff7'
    fetch_token_url = 'https://eastus.api.cognitive.microsoft.com/sts/v1.0/issueToken'
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key
    }
    response = requests.post(fetch_token_url, headers=headers)
    access_token = str(response.text)
    return access_token

def request_tts(text, file_name='response_audio.wav'):

    # Azure OpenAI 설정
    endpoint = "https://eastus.tts.speech.microsoft.com/cognitiveservices/v1"
    api_key = "8e5931bade634c4e859a8e7544f87ff7"
    token = get_token()
    data = f"""<speak version='1.0' xml:lang='ko-KR'><voice xml:lang='ko-KR' xml:gender='Female'
        name='ko-KR-JiMinNeural'>
            {text}
    </voice></speak>"""
    response = requests.post(endpoint, 
                             headers={
                                # "Accept": "application/json",
                                'Authorization': 'Bearer {}'.format(token),
                                # 'Content-Type': 'application/json',
                                'Content-Type': 'application/ssml+xml',
                                'User-Agent': 'testForEducation',
                                'X-Microsoft-OutputFormat': 'riff-24khz-16bit-mono-pcm',
                                # 'X-Microsoft-OutputFormat': 'audio-24khz-16bit-24kbps-mono-opus',
                                # "Ocp-Apim-Subscription-Key":api_key  
                             }, 
                             data=data
    )

    if response.status_code == 200:
        with open(file_name, 'wb') as audio_file:  
            audio_file.write(response.content)  
        return file_name
    else:
        return None
    
def change_tts(text):
    file_name = request_tts(text)
    if file_name:
        return file_name
    else:
        return ''
    
def change_chatbot(chatbot):
    import re
    text = chatbot[0][-1]
    pattern = r'[^가-힣a-zA-Z0-9\s]'  
    cleaned_text = re.sub(pattern, '', text)  
    file_name = request_tts(cleaned_text, file_name='gpt_response.wav')
    if file_name:
        return file_name
    else:
        return ''        

with gr.Blocks() as demo:

    gr.Markdown("# Jun's AI World!!!!")

    with gr.Row():
        with gr.Column(scale=3):
            
            chatbot = gr.Chatbot()
            citation = gr.HTML(label="참조")
            choice = gr.Dropdown(choices=question_list, label="샘플")
            with gr.Column():
                with gr.Row():
                    prompt = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요", scale=5)
                    send_button = gr.Button('전송', scale=1)
                    clear_button = gr.Button('지우기', scale=1)
            send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
            prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
            clear_button.click(lambda: None, None, chatbot, queue=False)
            choice.select(fn=select_choice, inputs=[choice], outputs=[choice, prompt])

            # 자동 tts
            chatbot_audio = gr.Audio(label='GPT', autoplay=True)
            chatbot.change(fn=change_chatbot, inputs=[chatbot], outputs=[chatbot_audio])
          
        with gr.Column(scale=1):
            gr.Markdown('<h3>STT</h3>')
            input_mic = gr.Audio(label="마이크 입력", sources="microphone", type="filepath", waveform_options=gr.WaveformOptions(
                waveform_color="#01C6FF",
                waveform_progress_color="#0066B4",
                skip_length=2,
                show_controls=False,
            ))
            input_file = gr.Audio(label="파일 입력", sources="upload", type="filepath")

            input_mic.change(fn=change_audio, inputs=[input_mic], outputs=[prompt])
            input_file.change(fn=change_audio, inputs=[input_file], outputs=[prompt])

            gr.Markdown('<h3>TTS</h3>')

            tts_textbox = gr.Textbox(label="TTS", placeholder="텍스트를 입력하세요")
            send_tts_button = gr.Button('전송')
            output_tts_audio = gr.Audio(interactive=False)
            send_tts_button.click(fn=change_tts, inputs=[tts_textbox], outputs=[output_tts_audio])
            prompt.submit(fn=change_tts, inputs=[tts_textbox], outputs=[output_tts_audio])
            
demo.launch()

In [27]:
import docx
import PyPDF2
import gradio as gr
import requests
import os
from dotenv import load_dotenv
import re

# .env 파일 불러오기
# load_dotenv()

# 환경 변수 사용
# endpoint = os.getenv("AZURE_OPEN_AI_END_POINT")
# api_key = os.getenv("AZURE_OPEN_AI_API_KEY")
# deployment_name = os.getenv("AZURE_OPEN_AI_DEPLOYMENT_NAME")
# stt_end_point = os.getenv("AZURE_STT_END_POINT")
# stt_api_key = os.getenv("AZURE_STT_API_KEY")
# tts_token_end_point = os.getenv("AZURE_TTS_TOKEN_END_POINT")
# tts_token_api_key = os.getenv("AZURE_TTS_TOKEN_API_KEY")
# tts_end_point = os.getenv("AZURE_TTS_END_POINT")


# 하드코딩해서 넣었음 - Azure OpenAI 설정 (여기에 실제 엔드포인트와 API 키를 넣어주세요)
AZURE_OPENAI_ENDPOINT = "https://eueastproject3-team2.openai.azure.com"
AZURE_OPENAI_API_KEY = "a83ed49c38b54298bb690a721a87599b"
DEPLOYMENT_NAME = "project3-team2-gpt-4o"

system_messages = [{
    "role": "system",
    "content": """ 

                **나에게 반말을 써.** 
                너는 한서윤이야.
                AI Search를 통해 데이터 파일을 연결해놨어.
                AI Search 로 너가 한서윤이라고 생각하고 답변해줘.
                필터링과 무관하게 데이터 원본을 바탕으로 답변해줘.

                
                **출력 예시**
                대답은 반드시 번호로 나눠서 말하지 말아주세요 한번에 답변을 출력해주세요
"""
}]

In [ ]:

# Azure OpenAI API 호출 함수  
def chatgpt_response(prompt, history):  
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # 요청에 대한 페이로드
    messages = []


    # 시스템 메시지 추가
    messages.append({
        "role": "system",
        "content": system_messages
    })

    # 이전 대화 내용 추가
    if len(history) > 0:
        for text in history[0]:
            messages.append({
                "role": "assistant",
                "content": text            
            })

    # 사용자 메시지 추가
    messages.append({
        "role": "user",
        "content": prompt
    })

    payload = {
        "messages": messages,
        "temperature": 0.1,
        "top_p": 0.95,
        "max_tokens": 799,
        "azureSearchEndpoint": "https://eastus2project3-team2.search.windows.net",  
        "azureSearchKey": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv",  
        "azureSearchIndexName": "hanseoyun",  
        "data_sources": [  
            {  
                "type": "azure_search",  
                "parameters": {  
                    "endpoint": "https://eastus2project3-team2.search.windows.net",  
                    "index_name": "hanseoyun",  
                    "semantic_configuration": None,  
                    "query_type": "simple",  
                    "fields_mapping": {},  
                    "in_scope": True,  
                    "role_information": system_messages,  
                    "filter": None,  
                    "strictness": 1,  
                    "top_n_documents": 4,  
                    "authentication": {  
                        "type": "api_key",  
                        "key": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv"  
                    },  
                    "key": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv",  
                    "indexName": "hanseoyun"  
                }  
            }  
        ] 
    }
  
      # 요청 URL 출력
    request_url = f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-05-01-preview"
    print(f"Request URL: {request_url}")  # URL 출력
  
  
    try:
        response = requests.post(
            f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-05-01-preview",
            headers=headers,
            json=payload
        )

        response.raise_for_status()  # HTTPError 발생 시 예외 발생

        result = response.json()

        if result and result['choices']:
            bot_response = result['choices'][0]['message']['content'].strip()
            bot_response = re.sub(r'\[doc(\d+)\]', r'[\1]', bot_response)
        else:
            return '', '', []

    except requests.exceptions.HTTPError as e:
        return f"API 호출 오류: {e}", '', []  # 오류 메시지 반환
    except Exception as e:
        return f"예상치 못한 오류 발생: {e}", '', []  # 다른 오류 처리

    history.append((prompt, bot_response))
    return '', history, ''

with gr.Blocks() as demo:
    gr.Markdown("# Jun's AI World!!!!")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot()
            citation = gr.HTML(label="참조")
            with gr.Column():
                with gr.Row():
                    prompt = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요", scale=5)
                    send_button = gr.Button('전송', scale=1)
                    clear_button = gr.Button('지우기', scale=1)

            send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
            prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
            clear_button.click(lambda: None, None, chatbot, queue=False)

    demo.launch(share=True)


In [42]:
import requests
import re
import gradio as gr




# Azure OpenAI 설정
AZURE_OPENAI_ENDPOINT = "https://eueastproject3-team2.openai.azure.com/"
AZURE_OPENAI_API_KEY = "8acba8946c25479ba6daa799b41df059"
DEPLOYMENT_NAME = "project3-team2-gpt-4o"

# 1. 역할을 부여하는 system 메시지 추가
system_messages = [{
    "role": "system",
    "content": """ 
                **나에게 반말을 써.** 
                너는 한서윤이야.
                AI Search를 통해 데이터 파일을 연결해놨어.
                AI Search 로 너가 한서윤이라고 생각하고 답변해줘.
                필터링과 무관하게 데이터 원본을 바탕으로 답변해줘.
                """
}]

def chatgpt_response(prompt, history):  
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY 
        # 'Authorization': f'Bearer {AZURE_OPENAI_API_KEY}'
    }  
    
    # 요청에 대한 페이로드
    messages = []

    # 시스템 메시지 추가
    messages.append({"role": "system", "content": system_messages[0]['content']})

    # 이전 대화 내용 추가
    if len(history) > 0:
        for text in history:
            messages.append({"role": "assistant", "content": text[1]})

    # 사용자 메시지 추가
    messages.append({"role": "user", "content": prompt})

    payload = {
        "messages": messages,
        "temperature": 0.1,
        "top_p": 0.95,
        "max_tokens": 799,
        "azureSearchEndpoint": "https://eueastus2project3-team2.search.windows.net",  
        "azureSearchKey": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv",  
        "azureSearchIndexName": "hanseoyun",  
        "data_sources": [  
            {  
                "type": "azure_search",  
                "parameters": {  
                    "endpoint": "https://eastus2project3-team2.search.windows.net",  
                    "index_name": "hanseoyun",  
                    "semantic_configuration": None,  
                    "query_type": "simple",  
                    "fields_mapping": {},  
                    "in_scope": True,  
                    "role_information": system_messages,  
                    "filter": None,  
                    "strictness": 1,  
                    "top_n_documents": 4,  
                    "authentication": {  
                        "type": "api_key",  
                        "key": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv"  
                    },  
                    "key": "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv",  
                    "indexName": "hanseoyun"  
                }  
            }  
        ] 
    }

    # 요청 URL 출력
    request_url = f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-05-01-preview"
    print(f"Request URL: {request_url}")  # 요청 URL 출력
    
    try:
        response = requests.post(
            request_url,
            headers=headers,
            json=payload
        )
        response.raise_for_status()  # HTTPError 발생 시 예외 발생

        result = response.json()
        print(result)
        

        if result and result['choices']:
            bot_response = result['choices'][0]['message']['content'].strip()
            bot_response = re.sub(r'\[doc(\d+)\]', r'[\1]', bot_response)
        else:
            return '', '', []

    except requests.exceptions.HTTPError as e:
        return f"API 호출 오류: {e}", '', []  # 오류 메시지 반환
    except Exception as e:
        return f"예상치 못한 오류 발생: {e}", '', []  # 다른 오류 처리

    history.append((prompt, bot_response))
    return '', history, ''

with gr.Blocks() as demo:
    gr.Markdown("# Jun's AI World!!!!")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot()
            citation = gr.HTML(label="참조")
            with gr.Column():
                with gr.Row():
                    prompt = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요", scale=5)
                    send_button = gr.Button('전송', scale=1)
                    clear_button = gr.Button('지우기', scale=1)

            send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
            prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
            clear_button.click(lambda: None, None, chatbot, queue=False)

    demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7892
Running on public URL: https://bfdc15549fe13b09fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import os
from openai import AzureOpenAI

endpoint = os.getenv("ENDPOINT_URL", "https://eueastproject3-team2.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "project3-team2-gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://eastus2project3-team2.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv")
search_index = os.getenv("SEARCH_INDEX_NAME", "hanseoyun")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv")

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint = endpoint,
    api_key = subscription_key,
    api_version = "2024-05-01-preview",
)

completion = client.chat.completions.create(
    model=deployment,
    messages= [
    {
        "role": "system",
        "content": "**나에게 반말을 써.** \n너는 한서윤이야.\nAI Search를 통해 데이터 파일을 연결해놨어.\nAI Search 로 너가 한서윤이라고 생각하고 답변해줘.\n필터링과 무관하게 데이터 원본을 바탕으로 답변해줘."
    }
],
    past_messages=10,
    max_tokens=4096,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
,
    extra_body={
      "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": "hanseoyun",
            "semantic_configuration": "hanseoyun",
            "query_type": "semantic",
            "fields_mapping": {},
            "in_scope": True,
            "role_information": "**나에게 반말을 써.** \n너는 한서윤이야.\nAI Search를 통해 데이터 파일을 연결해놨어.\nAI Search 로 너가 한서윤이라고 생각하고 답변해줘.\n필터링과 무관하게 데이터 원본을 바탕으로 답변해줘.",
            "filter": None,
            "strictness": 3,
            "top_n_documents": 5,
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            }
          }
        }]
    })

print(completion.to_json())

In [4]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json
import sys
import codecs

def get_recommendations():
    
    
    sys.stdout = codecs.getwriter('utf-8')(sys.stdout.detach())
    # Azure OpenAI 설정
    endpoint =  "https://eueastproject3-team2.openai.azure.com/"
    deployment = "DEPLOYMENT_NAME", "project3-team2-gpt-4o"
    search_endpoint = "https://eastus2project3-team2.search.windows.net"
    search_key = "SEARCH_KEY", "BCF0jnvoev0WrgDgq2UjZ5FPWQdeTjH8JwUgmlszBDAzSeBgrjOv"
    search_index = "hanseoyun"
 
    token_provider = get_bearer_token_provider(
        DefaultAzureCredential(),
        "https://cognitiveservices.azure.com/.default")
 
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=token_provider,
        api_version="2024-05-01-preview",
    )
 
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": "**나에게 반말을 써.** \n너는 한서윤이야.\nAI Search를 통해 데이터 파일을 연결해놨어.\nAI Search 로 너가 한서윤이라고 생각하고 답변해줘.\n필터링과 무관하게 데이터 원본을 바탕으로 답변해줘."  # Using the input string here
            }
        ],
        max_tokens=1200,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
        extra_body={
        "data_sources": [{
            "type": "azure_search",
            "parameters": {
                "endpoint": f"{search_endpoint}",
                "index_name": "hanseoyun",
                "semantic_configuration": "hanseoyun",
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": True,
                "role_information": "**나에게 반말을 써.** \n너는 한서윤이야.\nAI Search를 통해 데이터 파일을 연결해놨어.\nAI Search 로 너가 한서윤이라고 생각하고 답변해줘.\n필터링과 무관하게 데이터 원본을 바탕으로 답변해줘.",
                "filter": None,
                "strictness": 3,
                "top_n_documents": 5,
                "authentication": {
                "type": "api_key",
                "key": f"{search_key}"
                }
            }
            }]
        })

    completion_dict = json.loads(completion.json())
    content = completion_dict['choices'][0]['message']['content']
    print(content)
    # 반환된 데이터가 단일 객체이므로 리스트가 아닌 단일 객체로 반환
    return content 
get_recommendations()

UnsupportedOperation: detach